In [1]:
import pandas as pd
import numpy as np
import matplotlib

df = pd.read_csv('./files/Calib_SN04_test2.csv' ,delimiter=';',header=None)
df = df.rename(columns={0: 'ch', 1:'ADCmin', 2: 'DMMmin', 3:'ADCmax', 4:'DMMmax'})
df.head(10)

,ch,ADCmin,DMMmin,ADCmax,DMMmax
0,ch0,E6A4,108.752,F6F1,116.344
1,ch1,E6BD,108.756,F6CC,116.322
2,ch2,E6B1,108.740,F6C3,116.314
3,ch3,E6AC,108.740,F6BD,116.315
4,ch4,E6C1,108.744,F6CD,116.300
5,ch5,E6B8,108.741,F6C7,116.309
6,ch6,E6AB,108.736,F6BE,116.311
7,ch7,E6B6,108.741,F6C7,116.315
8,ch8,E6F7,108.748,F714,116.323
9,ch9,E6CE,108.712,F6E3,116.288


In [2]:
def hex_to_dec(df) :
    df = df.copy()
    df['ADCmin'] = df['ADCmin'].apply(lambda x: int(x,16) )
    df['ADCmax'] = df['ADCmax'].apply(lambda x: int(x,16) )
    return df 
    

In [3]:
df = hex_to_dec(df)
df.head()

,ch,ADCmin,DMMmin,ADCmax,DMMmax
0,ch0,59044,108.752,63217,116.344
1,ch1,59069,108.756,63180,116.322
2,ch2,59057,108.740,63171,116.314
3,ch3,59052,108.740,63165,116.315
4,ch4,59073,108.744,63181,116.300


In [4]:
def compute_gain(df, calib ):
    df = df.copy()
    if (calib == 'HV'):
        a_coeff = 1.8535e-3
        df['Gain HV'] = (df['DMMmax'] - df['DMMmin']) / (df['ADCmax'] - df['ADCmin'] ) * 1/a_coeff

    elif (calib == 'biasV'):
         a_coeff = 1.8409e-3
         df['Gain BiasV'] = (df['DMMmax'] - df['DMMmin']) / (df['ADCmax'] - df['ADCmin'] ) * 1/a_coeff

    else:
        a_coeff = 99999. 
        print('Non valid calib type')
        abort()


    return df

In [5]:
def compute_offset(df, calib):
    df = df.copy()
    if (calib == 'HV'):
        a_coeff = 1.8535e-3
        b_coeff = 0
        df['offset HV'] = df['ADCmin'] - (df['DMMmin'] - b_coeff )/ (df['Gain HV'] * a_coeff)

    elif (calib == 'biasV'):
        a_coeff = 1.8409e-3
        b_coeff = 0
        df['offset BiasV'] = df['ADCmin'] - (df['DMMmin'] - b_coeff )/ (df['Gain BiasV'] * a_coeff)

    else:
        a_coeff = 99999. 
        b_coeff = 99999.
        print('Non valid calib type')
        abort()

    return df

In [6]:
df = compute_gain(df, 'biasV')
df = compute_gain(df, 'HV')
df.head()

,ch,ADCmin,DMMmin,ADCmax,DMMmax,Gain BiasV,Gain HV
0,ch0,59044,108.752,63217,116.344,0.988275,0.981556
1,ch1,59069,108.756,63180,116.322,0.999744,0.992947
2,ch2,59057,108.740,63171,116.314,1.000071,0.993273
3,ch3,59052,108.740,63165,116.315,1.000446,0.993645
4,ch4,59073,108.744,63181,116.300,0.999151,0.992359


In [7]:
df = compute_offset(df, 'biasV')
df = compute_offset(df, 'HV')
df.head()

,ch,ADCmin,DMMmin,ADCmax,DMMmax,Gain BiasV,Gain HV,offset BiasV,offset HV
0,ch0,59044,108.752,63217,116.344,0.988275,0.981556,-732.356164,-732.356164
1,ch1,59069,108.756,63180,116.322,0.999744,0.992947,-23.772403,-23.772403
2,ch2,59057,108.740,63171,116.314,1.000071,0.993273,-7.742540,-7.742540
3,ch3,59052,108.740,63165,116.315,1.000446,0.993645,9.409901,9.409901
4,ch4,59073,108.744,63181,116.300,0.999151,0.992359,-48.274749,-48.274749


In [8]:
def convertADC(df ):
    df.copy()
    df['ADCmin_conv'] = df['ADCmin']* 1.8409e-3
    df['ADCmax_conv'] = df['ADCmax']* 1.8409e-3

    return df

In [9]:
df = convertADC(df)
df.head()

,ch,ADCmin,DMMmin,ADCmax,DMMmax,Gain BiasV,Gain HV,offset BiasV,offset HV,ADCmin_conv,ADCmax_conv
0,ch0,59044,108.752,63217,116.344,0.988275,0.981556,-732.356164,-732.356164,108.694100,116.376175
1,ch1,59069,108.756,63180,116.322,0.999744,0.992947,-23.772403,-23.772403,108.740122,116.308062
2,ch2,59057,108.740,63171,116.314,1.000071,0.993273,-7.742540,-7.742540,108.718031,116.291494
3,ch3,59052,108.740,63165,116.315,1.000446,0.993645,9.409901,9.409901,108.708827,116.280448
4,ch4,59073,108.744,63181,116.300,0.999151,0.992359,-48.274749,-48.274749,108.747486,116.309903


In [10]:
import math       
def get_USig_gain(df):
    df = df.copy()
    df['Gain_Usig HV'] = df['Gain HV'].apply(lambda x: math.ceil((x* 32768)))
    df['Gain_Usig BiasV'] = df['Gain BiasV'].apply(lambda x: math.ceil((x* 32768)))

    return df

def get_Sig_offset(df):
    df= df.copy()
    df['Offset_Sig HV'] = df['offset HV'].apply(lambda x: math.ceil(x) ) 
    df['Offset_Sig BiasV'] = df['offset BiasV'].apply(lambda x: math.ceil(x) ) 

    return df
    

In [11]:
df = get_USig_gain(df)
df = get_Sig_offset(df)

df.head()

,ch,ADCmin,DMMmin,ADCmax,DMMmax,Gain BiasV,Gain HV,offset BiasV,offset HV,ADCmin_conv,ADCmax_conv,Gain_Usig HV,Gain_Usig BiasV,Offset_Sig HV,Offset_Sig BiasV
0,ch0,59044,108.752,63217,116.344,0.988275,0.981556,-732.356164,-732.356164,108.694100,116.376175,32164,32384,-732,-732
1,ch1,59069,108.756,63180,116.322,0.999744,0.992947,-23.772403,-23.772403,108.740122,116.308062,32537,32760,-23,-23
2,ch2,59057,108.740,63171,116.314,1.000071,0.993273,-7.742540,-7.742540,108.718031,116.291494,32548,32771,-7,-7
3,ch3,59052,108.740,63165,116.315,1.000446,0.993645,9.409901,9.409901,108.708827,116.280448,32560,32783,10,10
4,ch4,59073,108.744,63181,116.300,0.999151,0.992359,-48.274749,-48.274749,108.747486,116.309903,32518,32741,-48,-48


In [12]:
def convert_to_exadec(df):
    df = df.copy()

    df['Gain_Usig BiasV'] = df['Gain_Usig BiasV'].apply(lambda x : hex(x) )
    df['Gain_Usig HV'] = df['Gain_Usig HV'].apply(lambda x : hex(x) )

    df['Offset_Sig BiasV'] = df['Offset_Sig BiasV'].apply(lambda x : hex(x) )
    df['Offset_Sig HV'] = df['Offset_Sig HV'].apply(lambda x : hex(x) )


    return df

In [13]:
df = convert_to_exadec(df)
df.head()

,ch,ADCmin,DMMmin,ADCmax,DMMmax,Gain BiasV,Gain HV,offset BiasV,offset HV,ADCmin_conv,ADCmax_conv,Gain_Usig HV,Gain_Usig BiasV,Offset_Sig HV,Offset_Sig BiasV
0,ch0,59044,108.752,63217,116.344,0.988275,0.981556,-732.356164,-732.356164,108.694100,116.376175,0x7da4,0x7e80,-0x2dc,-0x2dc
1,ch1,59069,108.756,63180,116.322,0.999744,0.992947,-23.772403,-23.772403,108.740122,116.308062,0x7f19,0x7ff8,-0x17,-0x17
2,ch2,59057,108.740,63171,116.314,1.000071,0.993273,-7.742540,-7.742540,108.718031,116.291494,0x7f24,0x8003,-0x7,-0x7
3,ch3,59052,108.740,63165,116.315,1.000446,0.993645,9.409901,9.409901,108.708827,116.280448,0x7f30,0x800f,0xa,0xa
4,ch4,59073,108.744,63181,116.300,0.999151,0.992359,-48.274749,-48.274749,108.747486,116.309903,0x7f06,0x7fe5,-0x30,-0x30
